In [1]:
import bs4 as bs
import pickle
import urllib
import time
import pandas as pd


In [2]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [3]:
import traceback

def get_soup(source_address, timeout=10):
    hdr = {'User-Agent': 'Mozilla/5.0'}
    
    try:
        req = urllib.request.Request(source_address, headers=hdr)
        source = urllib.request.urlopen(req, timeout=timeout).read()
        soup = bs.BeautifulSoup(source, 'lxml')
        return soup
    # 디버깅을 위해 추가 
    except urllib.error.URLError as e:
        print('Could not complete urllib request.')
        print(f"Error: {e.reason}")
        traceback.print_exc()
    except Exception as e:
        print('Could not create soup.')
        print(f"Error: {str(e)}")
        traceback.print_exc()
    
    return None

In [4]:
# pickle 파일 읽어오기
def read_pickle(file):
    with open(file, 'rb') as handle:
         load = pickle.load(handle)
    return load

# pickle 파일로 저장 
def to_pickle(var, dir):
    directory = dir + '.pickle'
    with open(directory, 'wb') as handle:
        pickle.dump(var, handle)
    return

In [5]:
# 링크 불러오기 
def get_links(query):
    # 검색어 띄어쓰기 처리 
    query = query.replace(' ', '%20')
    url = 'https://medium.com/search?q=' + query

    # TODO: chromedrive 위치 지정  
    path = "/Users/leeeunji/workspace_selenium/chromedriver"
    driver = webdriver.Chrome(path)
    driver.get(url)
    # action = ActionChains(driver)
    time.sleep(1)

    # 페이지 로드하는 버튼을 클릭하여 다음 포스트 불러오기 
    # TODO: range 변경하여 버튼 몇 번 클릭할지 지정
    for i in range(1, 10):
        try:
            xpath = f'//*[@id="root"]/div/div[3]/div[2]/div/main/div/div/div[2]/div[{10*i}]/div/div/button'
            element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, xpath)))
            element.click()
            time.sleep(1)
        except:
            print('Failed to click the button.')
            break
    
    time.sleep(2)

    # 현재 페이지 가져옴 
    source = driver.page_source
    soup = bs.BeautifulSoup(source, 'lxml')
    # TODO: 가져올 링크 주소를 가져올 태그의 요소로 지정
    #       medium 에서는 <a aria-label="Post Preview Title" href="우리가 가져올 주소">
    #       이렇게 되어 있어서 아래와 같이 가져온 것임 
    a_tags = soup.find_all('a', attrs={'aria-label': 'Post Preview Title'})
    links = []
    for a in a_tags:
        try:
            # medium 에서는 href 가 상대주소로 지정되어 있음 -> https://medium.com 앞에 추가 
            href = urllib.parse.quote(a['href'], safe=':/?&=')  # URL 인코딩
            full_url = "https://medium.com" + href
            # print(full_url) # URL 잘 받아왔는지 확인
            links.append(full_url)
        except Exception as e:
            print(f"Error processing URL: {a['href']}")
            print(e)
        
    driver.close()
    return links

In [6]:
def get_stories(links):
    stories = []
    # 우리가 구해놓은 URL로부터 본문 가져옴 
    for index, link in enumerate(links):
        try:
            # print(f'link: {link}') # 링크 잘 받았는지 확인
            soup = get_soup(link)
            # TODO: 링크의 본문을 가져올 태그와 클래스 지정
            #       medium 에서는 <p class="pw-post-body-paragraph ~~~"> 이 본문 
            #       아래와 같이 클래스에 'pw-post-body-paragraph'이 포함되면 가져옴  
            p_tags = soup.find_all('p', class_=lambda value: value and 'pw-post-body-paragraph' in value.split())
            # print(p_tags) # 태그들 잘 가져왔는지 확인
            story = ' '.join([p.get_text(strip=True) for p in p_tags])
            # print('Story:', story) # 태그들을 하나로 잘 뭉쳤는지 확인 
            stories.append(story)
            print(index + 1, 'of', len(links), 'stories') # 진행 상황 표시
        except:
            pass
    return stories

In [7]:
# TODO: 검색어 지정 (여러 개 리스트 형식으로)
# queries = ['개념', '기술', '파이썬', '자바', '인공지능', '깃허브']
queries = ['음식', '운동']
story_dic = dict.fromkeys(queries)

In [ ]:
for query in queries:
    print(' ::', query, ':: ')
    links = get_links(query)
    stories = get_stories(links)
    to_pickle(stories, 'data/medium_stories/' + query.replace(' ', '_'))
    story_dic[query] = stories

df = pd.DataFrame({col: pd.Series(stories) for col, stories in story_dic.items()})
df.columns = [col.replace(' ', '_') for col in df.columns]
# TODO: output 파일명 지정
df.to_csv('output_medium.csv')

In [9]:
# TODO: 간단하게 확인하고 싶은 pickle 파일의 파일명 지정
lst = read_pickle('data/medium_stories/운동.pickle')
lst[0]

'[커리어 노트 24]에서 자존감을 위한 마음 정리에 대해 얘기를 나누었다. 감정을 구체화해서 정신의 실체를 만나야 한다는데… 자신을 만나야 위로도 하고 치유도 한다는데… 다 좋은 말인 건 알겠는데 대체 어떻게 하라는 건지… 나 또한 답을 몰라 오랫동안답답하기만 했었다. 신체 건강을 위해서 우리는 어떤 일들을 하는지 살펴보자. - 주기적으로 건강검진을 받는다. 예방하고 초기에 발견하기 위해서다. - 아프면 병원에 간다. 전문가의 진단과 치료를 받기 위해서다.'